<a href="https://colab.research.google.com/github/intandeay/Applied-Data-Science-Capstone-Project/blob/main/Assignment_Week_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Descriptive Stats on Olympic Results

In this assignment, some questions that were proposed in the previous assignment are going to be tested. The dataset used is the Olympic Result & NOC.

The questions in the previous assignment are:
1. Is there any correlation between medalists and their previous achievements in Olympics?
2. Is there any significant difference between the age of medalists and non-medalist?
3. What sports contribute the most medals for each country?


## Summary of The Different Descriptive Statistics

In [1]:
# Importing necessary libraries and modules

import pandas as pd
import numpy as np
import scipy.stats as stats

In [3]:
# Read The Dataset
df = pd.read_excel('/content/athlete_events_sep.xlsx')
noc = pd.read_csv("/content/noc_regions.csv")

In [4]:
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 16 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   ID       271116 non-null  int64  
 1   Name     271116 non-null  object 
 2   Sex      271116 non-null  object 
 3   Age      261642 non-null  float64
 4   Height   210945 non-null  float64
 5   Weight   208241 non-null  float64
 6   Team     271116 non-null  object 
 7   NOC      271116 non-null  object 
 8   Games    271116 non-null  object 
 9   Year     271116 non-null  int64  
 10  Season   271116 non-null  object 
 11  City     271116 non-null  object 
 12  Sport    271116 non-null  object 
 13  Event    271116 non-null  object 
 14  Medal    39783 non-null   object 
 15  isMedal  271116 non-null  bool   
dtypes: bool(1), float64(3), int64(2), object(10)
memory usage: 31.3+ MB


### Is There Any Correlation Between Medalists and Their Previous Achievements in Olympics?

In this part, first we are going to do some cumulative sum of the medals for each athlete. 

In [5]:
# Create new binary column to describe get medal/not
df['isMedal'] = df['Medal'].notnull()
# New column for cumulative sum of medals
df['sumMedal'] = 0

In [12]:
# Iterating per each row
for i, row in df.iterrows():
    ath_id = row['ID']
    year = row['Year']

    # Get the previous achievements for athlete
    ath_prev_performance = df.loc[(df['ID'] == ath_id) & (df['Year'] < year)]
    # Cumulative sum
    prev_medal = ath_prev_performance['isMedal'].sum()
    df.loc[i, 'sumMedal'] = prev_medal

In [17]:
# Binary variable to describe whether the athlete 
# has won any medal previously in specific Olympic Games
df['prevGotMedal'] = df['sumMedal'] > 0

Now, there are two binary variables that we are going to explore their correlation, isMedal and prevGotMedal. IsMedal feature signifies that the athlete won a medal in a specific Olympic Games. Meanwhile, prevGotMedal means to describe whether the athlete has got medals before that Olympic Games. Since these two variables are binary, hence we are going to use Chi-Square Test.

To perform Chi-Square Test, first we have to count the contingency table/crosstab for each combination of isMedal and prevGotMedal value. There will be four combinations.

In [24]:
ct = pd.crosstab(index=df['isMedal'], columns=df['prevGotMedal'],margins=True, margins_name="Total")

In [25]:
ct

prevGotMedal,False,True,Total
isMedal,,,
False,216584,14749,231333
True,31308,8475,39783
Total,247892,23224,271116


For this Ch-Square Test, here are the hypotheses

- H0: The two features have no relationship
- H1: The two features does have any relationship

To know more about the Chi-square test, here is the reference I used to calculate it: https://towardsdatascience.com/chi-square-test-with-python-d8ba98117626

In [26]:
# Calcualtion of Chisquare
chi_square = 0
rows = df['isMedal'].unique()
columns = df['prevGotMedal'].unique()
for i in columns:
    for j in rows:
        O = ct[i][j]
        E = ct[i]['Total'] * ct['Total'][j] / ct['Total']['Total']
        chi_square += (O-E)**2/E

In [30]:
# The p-value approach
alpha = 0.05

print("Approach 1: The p-value approach to hypothesis testing in the decision rule")
p_value = 1 - stats.chi2.cdf(chi_square, (len(rows)-1)*(len(columns)-1))
conclusion = "Failed to reject the null hypothesis."
if p_value <= alpha:
    conclusion = "Null Hypothesis is rejected."
        
print("chisquare-score is:", chi_square, " and p value is:", np.round(p_value, 4))
print(conclusion)
    
# The critical value approach
print("\n--------------------------------------------------------------------------------------")
print("Approach 2: The critical value approach to hypothesis testing in the decision rule")
critical_value = stats.chi2.ppf(1-alpha, (len(rows)-1)*(len(columns)-1))
conclusion = "Failed to reject the null hypothesis."
if chi_square > critical_value:
    conclusion = "Null Hypothesis is rejected."
        
print("chisquare-score is:", chi_square, " and critical value is:", critical_value)
print(conclusion)

Approach 1: The p-value approach to hypothesis testing in the decision rule
chisquare-score is: 9657.389481764021  and p value is: 0.0
Null Hypothesis is rejected.

--------------------------------------------------------------------------------------
Approach 2: The critical value approach to hypothesis testing in the decision rule
chisquare-score is: 9657.389481764021  and critical value is: 3.841458820694124
Null Hypothesis is rejected.


By the Chi-square Test, we see that the p-value is less than 0.05. This means that there is enough evidence that there is any relationship between achievement of athletes in Olympic Games and their previous results.

## Is There Any Significant Difference Between The Age of Medalists and Non-Medalist?

In this part, we are going to explore whether age has any connection for performance. First, we are going to extract the ages of each type of medal. Then we also extract the age of any medalists and non-medalists.

In [48]:
age_gold = df[(df['Medal'] =='Gold') & (df['Age'].notnull())]['Age']
age_silver = df[(df['Medal'] =='Silver') & (df['Age'].notnull())]['Age']
age_bronze = df[(df['Medal'] =='Bronze') & (df['Age'].notnull())]['Age']
age_nonmedal = df[(df['Medal'].isnull()) & (df['Age'].notnull())]['Age']
age_medal = df[(df['Medal'].notnull()) & (df['Age'].notnull())]['Age']

In [54]:
print("Mean Age of Gold Medalist", age_gold.mean())
print("Mean Age of Silver Medalist", age_silver.mean())
print("Mean Age of Bronze Medalist", age_bronze.mean())
print("Mean Age of Medalist", age_medal.mean())
print("Mean Age of Non-Medalist", age_nonmedal.mean())

Mean Age of Gold Medalist 25.901013309134907
Mean Age of Silver Medalist 25.99672412448327
Mean Age of Bronze Medalist 25.879209595571275
Mean Age of Medalist 25.925174771452717
Mean Age of Non-Medalist 25.492288547155994


#### Gold Medalist vs Non-Medalist

T-test for two independent samples is used.

In [49]:
stats.ttest_ind(age_gold, age_nonmedal)

Ttest_indResult(statistic=7.089918154207331, pvalue=1.345654555600641e-12)

Since p-value is < 0.05, we can conclude that there is enough evidence to support that ages of gold medalists are significantly different than non-medalist

#### Gold vs Silver vs Bronze

For this test, since there are more than two groups, hence we use One way ANOVA Test

In [51]:
stats.f_oneway(age_gold, age_silver, age_bronze)

F_onewayResult(statistic=1.4415127134142867, pvalue=0.23658221471061605)

Since p-value is > 0.05, we can conclude that there is not enough evidence to support that ages of each type of medalists have significant different among each other.

#### Medalists vs Non-Medalists

For this test, T-test for two independent samples is used.

In [52]:
stats.ttest_ind(age_medal, age_nonmedal)

Ttest_indResult(statistic=12.344477668888402, pvalue=5.335730124331498e-35)

Since p-value is < 0.05, we can conclude that there is enough evidence to support that ages of medalists are significantly different than non-medalists

### Countries and Sports that Contribute the Most Medals

In this part, we are going to extract the name of the sports that contribute the most medals for each country. For this, we are going to use groupby and aggregation by mode in Sport column.

In [68]:
# Only medal
df_medal = df[df['isMedal']]

# Perform Group by
country_sportMostMedals = df_medal.groupby('NOC').agg({'Sport':pd.Series.mode}).reset_index()
# Merge with NOC dataset to get the country/region name
country_sportMostMedals = country_sportMostMedals.merge(noc, on = 'NOC', how = 'inner')

for i, r in country_sportMostMedals.iterrows():
    print("Country:", r['region'], "| Sport:", r['Sport'])

Country: Afghanistan | Sport: Taekwondo
Country: Curacao | Sport: Sailing
Country: Algeria | Sport: Athletics
Country: Australia | Sport: Rugby
Country: Argentina | Sport: Hockey
Country: Armenia | Sport: Wrestling
Country: Australia | Sport: Swimming
Country: Austria | Sport: Alpine Skiing
Country: Azerbaijan | Sport: Wrestling
Country: Bahamas | Sport: Athletics
Country: Barbados | Sport: Athletics
Country: Burundi | Sport: Athletics
Country: Belgium | Sport: Water Polo
Country: Bermuda | Sport: Boxing
Country: Belarus | Sport: Canoeing
Country: Czech Republic | Sport: Fencing
Country: Botswana | Sport: Athletics
Country: Brazil | Sport: Football
Country: Bahrain | Sport: Athletics
Country: Bulgaria | Sport: Wrestling
Country: Canada | Sport: Ice Hockey
Country: Chile | Sport: Football
Country: China | Sport: Gymnastics
Country: Ivory Coast | Sport: Taekwondo
Country: Cameroon | Sport: Football
Country: Colombia | Sport: ['Cycling' 'Weightlifting']
Country: Costa Rica | Sport: Swimmi

In [71]:
country_sportMostMedals.Sport.value_counts()

TypeError: ignored

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 5231, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'numpy.ndarray'


Athletics                               32
Boxing                                  12
Wrestling                               11
Football                                 7
Rowing                                   6
Hockey                                   6
Fencing                                  5
Gymnastics                               5
Ice Hockey                               5
Taekwondo                                4
Handball                                 4
Shooting                                 4
Swimming                                 4
Canoeing                                 3
Sailing                                  3
Water Polo                               3
Weightlifting                            3
Art Competitions                         2
Judo                                     2
Badminton                                2
Basketball                               2
Baseball                                 2
Alpine Skiing                            2
Rugby      

## Key Points That I Discovered

1. It is supported with evidence that previous performances in Olympic Games has an association to the performance in the next Olympic Games.
2. Between three type of medalists, there are no strong evidence that their ages are significantly different. But, between medalists and non-medalists, there are strong evidence that their ages are significantly different.
3. The mean of medalists' ages are six months older than the mean of non-medalists' ages.
4. Each country has their own most medal-contributing sports. But among these sports, atheletics, boxing, and wrestling are the sports you can find most often as the most medal-contributing sports.

## Did you prove or disprove any of your initial hypotheses? If so, which one and what do you plan to do next?

In my previous assignment, my initial hypotheses are:
1. There is enough evidence to support that previous achievement has any association to current performance in Olympic Games.
2. There is no strong evidence to support that ages between medalists and non-medalists are different.
3. Type of sport will depend on the country’s strengths in the sport

Two of my hypotheses are right, and this is what I will do:
1. Dig deeper on whether number of previous medals or type of medals will have any connection to the achievement in the next Olympic Games.
2. Since medalists are 6 months older than non-medalists, my thought is that this is due to those who were born in the first half of the year can enter or be accepted in earlier registration of training that led to more experience for the athlete. Then, there should be some way to compensate this for the ones who were born in the second half of the year.
3. I would like to recommend the result to Olympic Committee for each country so that they can set the target as best as possible.

## What additional questions are you seeking to answer?

1. Are the dominating countries in Winter Games are different than the Summer Games? In terms of number of medals
2. Is there any connection between the continent where the Games held and where the athletes come from, in terms of medal achievement?